In [ ]:
import random
import sys
import rtsvg
rt = rtsvg.RACETrack()
#_svg_ = rt.rayIntersectsSegmentSVG((0.7,0),(0.9,0.8),(1,0),(0.8,0.8))
#rt.tile([_svg_]) 

In [ ]:
x = lambda: random.uniform(-1.0, 1.0)
_tiles_ = []
for i in range(16): _tiles_.append(rt.rayIntersectsSegmentSVG((x(),x()),(x(),x()),(x(),x()),(x(),x()), w=128, h=128))
rt.table(_tiles_, per_row=16, spacer=10)

In [ ]:
# Works reliably when using p0 as the end point
_tiles_ = []
for i in range(16):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    _uv_       = (_p0_[0]-_xy_[0], _p0_[1]-_xy_[1])
    _tiles_.append(rt.rayIntersectsSegmentSVG(_xy_,_uv_,_p0_, _p1_, w=128, h=128))
rt.table(_tiles_, per_row=16, spacer=10)

In [ ]:
# Fails intermitently when using p1 as the end point
_tiles_ = []
for i in range(16):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    _uv_       = (_p1_[0]-_xy_[0], _p1_[1]-_xy_[1])
    _tiles_.append(rt.rayIntersectsSegmentSVG(_xy_,_uv_,_p0_, _p1_, w=128, h=128))
rt.table(_tiles_, per_row=16, spacer=10)

In [ ]:
# Measure the accuracy of the rayIntersectsSegment function
_correct_, _incorrect_ = 0, 0
for i in range(1_000_000):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    _uv_       = (_p1_[0]-_xy_[0], _p1_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_)
    if _xy_inter_ is not None: _correct_   += 1
    else:                      _incorrect_ += 1
print(f"Correct: {_correct_} Incorrect: {_incorrect_} | {_correct_/(_incorrect_+_correct_)}") # 78% correct rate for 10M iterations

In [ ]:
# Measure the accuracy of the rayIntersectsSegment function w/ slight epsilon
_correct_, _incorrect_, _ep_ = 0, 0, 10e-9 # around 10e-12, you start seeing the issue... (1 incorrect out of 1M)
# _ep_ = sys.float_info.min # smallest possible float ... you get about 78% correct rate here
for i in range(1_000_000):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    _p_        = (_p1_[0] - _ep_ * (_p1_[0] - _p0_[0]), _p1_[1] - _ep_ * (_p1_[1] - _p0_[1]))
    _uv_       = (_p_[0]-_xy_[0], _p_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_)
    if _xy_inter_ is not None: _correct_   += 1
    else:                      _incorrect_ += 1
print(f"Correct: {_correct_} Incorrect: {_incorrect_} | {_correct_/(_incorrect_+_correct_)}") # 78% correct rate for 10M iterations

In [ ]:
def rayIntersectsSegment(self, xy_ray, uv_ray, xy0_segment, xy1_segment, include_xy1_endpoint=False, epsilon=1e-9):
    x_r,  y_r  = xy_ray
    dx_r, dy_r = uv_ray
    x0,   y0   = xy0_segment
    x1,   y1   = xy1_segment    
    # Segment direction vector
    dx_s, dy_s = x1 - x0, y1 - y0
    # Compute determinant
    det = -dx_r * dy_s + dy_r * dx_s
    if abs(det) < 1e-10: return None # Lines are parallel or collinear
    # Compute parameters t and u
    t = ((x_r - x0) * dy_s - (y_r - y0) * dx_s) / det
    u = ((x_r - x0) * dy_r - (y_r - y0) * dx_r) / det
    # Check if intersection is valid (t >= 0 for ray, 0 <= u <= 1 for segment)
    if t >= 0.0:
        if include_xy1_endpoint:
            if 0.0 <= u <= 1.0+epsilon: return (x_r + t * dx_r, y_r + t * dy_r)
        else:
            if 0.0 <= u <  1.0-epsilon: return (x_r + t * dx_r, y_r + t * dy_r)
    return None

case1_correct, case1_incorrect = 0, 0
case2_correct, case2_incorrect = 0, 0
case3_correct, case3_incorrect = 0, 0
case4_correct, case4_incorrect = 0, 0
case5_correct, case5_incorrect = 0, 0
case6_correct, case6_incorrect = 0, 0

_ep_ = 1e-8 # starts falling apart if smaller now... so 1e-9 has errors...
for i in range(1_000_000):
    _xy_       = (x(),x())
    _p0_, _p1_ = (x(),x()), (x(),x())
    # The P1 EndPoint (endpoint flag matters here)
    _p_        = _p1_
    _uv_       = (_p_[0]-_xy_[0], _p_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=False)
    if _xy_inter_ is None:     case1_correct   += 1
    else:                      case1_incorrect += 1    
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=True)
    if _xy_inter_ is not None: case2_correct   += 1
    else:                      case2_incorrect += 1

    # The P1 EndPoint with epsilon (the endpoint flag shouldn't matter here - both should return an intersection)
    _p_        = (_p1_[0] - _ep_ * (_p1_[0] - _p0_[0]), _p1_[1] - _ep_ * (_p1_[1] - _p0_[1]))
    _uv_       = (_p_[0]-_xy_[0], _p_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=False)
    if _xy_inter_ is None: case3_incorrect += 1
    else:                  case3_correct   += 1
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=True)
    if _xy_inter_ is None: case4_incorrect += 1
    else:                  case4_correct   += 1

    # The P1 EndPoint with added epsilon (the endpoint flag shouldn't matter here -- neither should return an intersection)
    _p_        = (_p1_[0] + _ep_ * (_p1_[0] - _p0_[0]), _p1_[1] + _ep_ * (_p1_[1] - _p0_[1]))
    _uv_       = (_p_[0]-_xy_[0], _p_[1]-_xy_[1])
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=False)
    if _xy_inter_ is None: case5_correct   += 1
    else:                  case5_incorrect += 1
    _xy_inter_ = rt.rayIntersectsSegment(_xy_,_uv_,_p0_, _p1_, include_xy1_endpoint=True)
    if _xy_inter_ is None: case6_correct   += 1
    else:                  case6_incorrect += 1

print(f"Case 1: Correct: {case1_correct:8} Incorrect: {case1_incorrect:8} | {case1_correct/(case1_incorrect+case1_correct):.2f}")
print(f"Case 2: Correct: {case2_correct:8} Incorrect: {case2_incorrect:8} | {case2_correct/(case2_incorrect+case2_correct):.2f}")
print(f"Case 3: Correct: {case3_correct:8} Incorrect: {case3_incorrect:8} | {case3_correct/(case3_incorrect+case3_correct):.2f}")
print(f"Case 4: Correct: {case4_correct:8} Incorrect: {case4_incorrect:8} | {case4_correct/(case4_incorrect+case4_correct):.2f}")
print(f"Case 5: Correct: {case5_correct:8} Incorrect: {case5_incorrect:8} | {case5_correct/(case5_incorrect+case5_correct):.2f}")
print(f"Case 6: Correct: {case6_correct:8} Incorrect: {case6_incorrect:8} | {case6_correct/(case6_incorrect+case6_correct):.2f}")